In [ ]:
#!pip install -U numpy
#!pip install freetype=2.10.4
#%matplotlib
#!pip install torchvision

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import random

# Carregar os dados do FashionMNIST
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.FashionMNIST(
    root='./data_v3_02', 
    train=True, 
    download=True, 
    transform=transform
)


# Determinar a porcentagem dos dados a serem usados
percent_to_use = 0.3  # Por exemplo, usar 30% dos dados

# Calcular o número de exemplos correspondentes à porcentagem desejada
num_to_use = int(len(trainset) * percent_to_use)

# Criar uma lista de índices dos dados
indices = list(range(len(trainset)))

# Fazer a amostragem aleatória dos índices
sampled_indices = random.sample(indices, num_to_use)

# Criar um DataLoader com a amostra aleatória


trainloader = torch.utils.data.DataLoader(
    trainset, 
    batch_size=64, 
    #shuffle=True,
    sampler=torch.utils.data.SubsetRandomSampler(sampled_indices)
)

testset = torchvision.datasets.FashionMNIST(
    root='./data_v3_02', 
    train=False, 
    download=True, 
    transform=transform
)


# Determinar a porcentagem dos dados a serem usados
percent_to_use_test = 0.3  # Por exemplo, usar 30% dos dados

# Calcular o número de exemplos correspondentes à porcentagem desejada
num_to_use_test = int(len(testset) * percent_to_use_test)

# Criar uma lista de índices dos dados
indices_test = list(range(len(testset)))

# Fazer a amostragem aleatória dos índices
sampled_indices_test = random.sample(indices_test, num_to_use_test)


testloader = torch.utils.data.DataLoader(
    testset, 
    batch_size=64, 
    #shuffle=False,
    sampler=torch.utils.data.SubsetRandomSampler(sampled_indices_test)
)

# Definir a arquitetura do MLP
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instanciar o modelo
model = MLP()

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Função para calcular a acurácia
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    return correct / total


# Treinamento do modelo
num_epochs = 2
train_losses = []
test_losses = []
accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(trainloader)
    train_losses.append(train_loss)

    model.eval()
    test_loss = 0.0
    accuracy_test = 0.0
    for inputs, labels in testloader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        accuracy_test += accuracy(outputs, labels)
    test_loss /= len(testloader)
    test_losses.append(test_loss)
    accuracy_test /= len(testloader)
    accuracies.append(accuracy_test)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy_test:.2%}')


# Plotar os resultados
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.show()


plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), accuracies, label='Test Accuracy', color='green')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Test Accuracy')
plt.legend()
plt.show()



Epoch [1/2], Train Loss: 0.6308, Test Loss: 0.4737, Test Accuracy: 81.94%
Epoch [2/2], Train Loss: 0.4497, Test Loss: 0.4800, Test Accuracy: 81.71%
